# How is heart disease linked to the patient's age, maximum heart rate during strenuous exercise, and ST depression induced by exercise?

## Introduction

Heart attacks often roots from a cardiovascular disease and is induced by a blockage in the arteries, restricting blood flow to heart muscles. Heart attacks may be fatal if the condition is not immediately treated. Thus, it is important to identify a heart condition early on and prevent sudden and unexpected heart attacks. 

This study aims to predict the likelihood of a patient developing heart disease by conducting classification with K algorithm. The Heart Attack dataset used compares 13 variables that are possibly linked to heart disease, with whether or not they developed the disease. To classify whether the patient has a heart disease, the study uses 3 variables: age, maximum heart rate, and ST depression induced by exercise. Based on previous studies, people ages 65 and older are much more likely to suffer a heart attack. In addition, a low maximum heart (exercise intolerance) and ST depression (when ST segment on ECG is abnormally low) are often the first symptoms of heart disease. 

Thus, the question being addressed is: can heart disease be accurately predicted based on a patient’s age, maximum heart rate during strenuous exercise, and ST depression induced by exercise?
